# 4. 좋은 훈련 데이터셋 만들기:파이토치

데이터의 품질과 데이터에 담겨있는 유용한 정보의 양은 머신러닝 알고리즘을 얼마나 잘 학습 시킬 수 있는지 결정하는 중요한 요소이다. 이 노트북에서는 데이터셋을 다루는 방법을 배우고, 머신러닝 알고리즘에 데이터를 효과적으로 제공할 수 있는 방법을 배운다. ((배워보자 ...))

* 누락된 데이터 다루기
* 범주형 데이터 다루기
* 데이터셋을 훈련 데이터셋과 테스트 데이터셋으로 나누기
* 특성 스케일 맞추기
* 유용한 특성 선택하기
* 랜덤 포레스트 특성 중요도 사용
...

## 누락된 데이터 다루기

누락된 데이터는 다루기 까다로운 문제이다. 누락된 데이터가 있는 경우에는 해당 데이터를 제거하거나, 채워넣어야 한다. 누락된 데이터를 제거하는 것은 간단하지만, 데이터가 많은 경우에는 데이터를 제거할 수 없다. 누락된 데이터를 채워넣는 방법은 여러가지가 있는데, 평균값, 중간값, 최빈값 등으로 채워넣을 수 있다.

NaN 값등 누락된 데이터를 다루기 위해서는 `SimpleImputer`를 사용할 수 있다. `SimpleImputer`는 누락된 데이터를 채워넣는데 사용된다. `SimpleImputer`는 누락된 데이터를 특정 값으로 채워넣는데 사용된다. `SimpleImputer`는 `strategy` 매개변수를 사용하여 누락된 데이터를 채워넣는 방법을 지정할 수 있다. `strategy` 매개변수의 값으로는 `mean`, `median`, `most_frequent`, `constant` 등이 있다.

### 4.1.1. 테이블 형태 데이터에서 누락된 값 식별하기

In [5]:
import pandas as pd

from io import StringIO

csv_data = \
  """A,B,C,D
  1.0,2.0,3.0,4.0
  5.0,6.0,,8.0
  10.0,11.0,12.0
  """
  
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


누락된 값은`NaN`으로 표시된다. `NaN`은 `numpy` 모듈에 정의되어 있다.

In [6]:
# 각 열마다 누락된 값의 개수를 출력
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### 4.1.2. 누락된 값이 있는 훈련 샘플이나 특성 제거하기

누락된 값이 있는 훈련 샘플이나 특성을 제거하는 방법은 간단하다. `dropna` 메서드를 사용하여 누락된 값이 있는 훈련 샘플이나 특성을 제거할 수 있다.

In [7]:
df.dropna(axis=0) # 행 삭제

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [8]:
df.dropna(axis=1) # 열 삭제

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [9]:
# 모든 열이 NaN인 행만 삭제
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [10]:
# NaN이 아닌 값이 네 개보다 작은 행 삭제
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [11]:
# 특정 열에 NaN이 있는 행만 삭제
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


4.1.3. 누락된 값 대체하기

누락된 값을 삭제하지 못할 때 대체하는 방법이 있다. `SimpleImputer`를 사용하여 누락된 값을 대체할 수 있다. `SimpleImputer`는 누락된 값을 대체하는데 사용된다. `SimpleImputer`는 `strategy` 매개변수를 사용하여 누락된 값을 대체하는 방법을 지정할 수 있다. `strategy` 매개변수의 값으로는 `mean`, `median`, `most_frequent`, `constant` 등이 있다.

In [15]:
import numpy as np

from sklearn.impute import SimpleImputer

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
print(pd.DataFrame(imputed_data))
print(df)

      0     1     2    3
0   1.0   2.0   3.0  4.0
1   5.0   6.0   7.5  8.0
2  10.0  11.0  12.0  6.0
      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN


* [C:1]은 평균값 7.5로 누락된 값을 대체했다.
* [D:2]는 평균값 6으로 누락된 값을 대체했다.

`startegy` 매개변수를 설정할 수 있는 다름 값으로는 `median`, `most_frequent`, `constant` 등이 있다.

* `median`: 중간값으로 누락된 값을 대체한다.
* `most_frequent`: 가장 많이 나타난 값으로 대체한다.
* `constant`: `fill_value` 매개변수에 지정된 값으로 누락된 값을 대체한다.

### 4.1.4. 사이킷런의 추정기 API

`SimpleImputer`는 사이킷런의 변환기이다. 변환기는 데이터를 변환하는 추정기를 말한다. 변환기는 `fit` 메서드와 `transform` 메서드를 가지고 있다. `fit` 메서드는 훈련 데이터를 사용하여 변환 매개변수를 학습하고, `transform` 메서드는 학습한 매개변수를 사용하여 데이터를 변환한다. `SimpleImputer`는 `fit` 메서드를 사용하여 훈련 데이터의 평균값, 중간값, 최빈값 등을 계산하고, `transform` 메서드를 사용하여 누락된 값을 대체한다.

## 4.2. 범주형 데이터 다루기

범주형 데이터를 다룰 때,

* 순서가 있는 것과 없는 것을 구분해야한다.
  * 순서가 있는 특성은 정렬하거나 차례대로 놓을 수 있는 범주형 특성으로 생각할 수 있다.
  * 반대로 순서가 없는 특성은 차례를 부여할 수 없다.

### 4.2.1. 판다스에서 범주형 데이터 다루기

In [19]:
import pandas as pd

df = pd.DataFrame([
  ['green', 'M', 10.1, 'class2'],
  ['red', 'L', 13.5, 'class1'],
  ['blue', 'XL', 15.3, 'class2']
], columns=['color', 'size', 'price', 'classlabel'])

df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


* `color`는 순서가 없는 특성이다.
* `size`는 순서가 있는 특성이다.
* `price`는 수치형 특성이다.
* `classlabel`는 라벨?

`size`의 순서를 자동으로 인식시키키 웨해서 매핑 함수를 만들어야 한다.

수식$XL = L + 1 = M + 2$을 가정한다고 하자.

In [20]:
size_mapping = {
  'XL': 3,
  'L': 2,
  'M': 1
}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [21]:
# 거꾸로 매핑
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### 4.2.3. 클래스 레이블 인코딩

In [22]:
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [25]:
# 매핑 딕셔너리를 사용하여 클래스 레이블을 정수로 변환
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [24]:
# 거꾸로 매핑
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [29]:
# 사이킷런의 LabelEncoder를 사용하여 클래스 레이블을 정수로 변환
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

* `fit_transform` 메서드는 `fit` 메서드와 `transform` 메서드를 합쳐놓은 단축 메서드 이다.
* `inverse_transform` 메서드는 `transform` 메서드로 변환된 데이터를 원래의 값으로 되돌린다.

In [28]:
class_le.inverse_transform(y)

array([1, 0, 1])

### 4.2.4. 순서가 없는 특성에 원-핫 인코딩 적용하기

`OneHotEncoder`는 순서가 없는 특성을 원-핫 인코딩하는데 사용된다. `OneHotEncoder`는 `fit` 메서드와 `transform` 메서드를 가지고 있다. `fit` 메서드는 훈련 데이터를 사용하여 변환 매개변수를 학습하고, `transform` 메서드는 학습한 매개변수를 사용하여 데이터를 변환한다.


In [30]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

`color`는 순서가 없는데 `blue:0`, `green:1`, `red:2`로 매핑되었다. 이대로 분류기에 넣으면 분류기는 `red`가 `green`보다 `blue`보다 크다고 생각할 것이다. 이러한 문제를 해결하기 위해서 `OneHotEncoder`를 사용하여 순서가 없는 특성을 원-핫 인코딩할 수 있다.

In [31]:
from sklearn.preprocessing import OneHotEncoder
X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

배열의 다른 두 열을 수정하지 않기 위해 `OneHotEncoder`를 하나의 열 (X[:, 0])에만 적용했다. `OneHotEncoder`는 기본적으로 희소 행렬을 반환한다. `toarray` 메서드를 사용하여 밀집 배열로 변환할 수 있다. 여러 개의 특성이 있는 배열에서 특정 열만 반환 하려면 `ColumnTransformer`를 사용할 수 있다.
이 클래스는 (name, transformer, columns) 튜플의 리스트를 입력으로 받아서 각 열에 다른 변환을 적용할 수 있다.

In [34]:
from sklearn.compose import ColumnTransformer
X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([
  ('onehot', OneHotEncoder(dtype=int), [0]), # 0번째 열에 대해 OneHotEncoder 적용
  ('nothing', 'passthrough', [1, 2]) # 1, 2번째 열은 그대로 유지
])

c_transf.fit_transform(X)

array([[0, 1, 0, 1, 10.1],
       [0, 0, 1, 2, 13.5],
       [1, 0, 0, 3, 15.3]], dtype=object)

*원-핫 인코딩된 데이터셋을 사용할 때* 다중 공선성(multicollinearity) 문제가 발생할 수 있다. 다중 공선상 이란 두 개 이상의 특성이 서로 상관관계가 있을 때 발생하는 문제이다. 다중 공선성 문제를 해결하기 위해서 원-핫 인코딩된 특성 중 하나를 삭제한다. `OneHotEncoder`는 `drop` 매개변수를 사용하여 원-핫 인코딩된 특성 중 하나를 삭제할 수 있다. `drop` 매개변수의 값으로는 `first`, `if_binary` 등이 있다.

`get_dummies`를 사용할 때 `drop_first` 매개변수를 `True`로 지정하여 첫번째 열을 삭제할 수 있다.


In [37]:
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True, dtype=int)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


`OneHotEncoder`에서 중복열을 삭제하려면 `drop='first'`와 `categories='auto'`를 사용한다.

In [38]:
color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([
  ('onehot', color_ohe, [0]),
  ('nothing', 'passthrough', [1, 2])
])
c_transf.fit_transform(X)

array([[1.0, 0.0, 1, 10.1],
       [0.0, 1.0, 2, 13.5],
       [0.0, 0.0, 3, 15.3]], dtype=object)

**순서가 있는 특성 인코딩하기**

순서가 있는 특성의 범주 사이에서 수치적 크기에 대해 확신할 수 없거나, 두 범주 사이의 순서를 정의할 수 없다면 임계 값을 사용하여 0/1로 인코딩 할 수 있다.

In [1]:
import pandas as pd

df = pd.DataFrame([
  ['green', 'M', 10.1, 'class2'],
  ['red', 'L', 13.5, 'class1'],
  ['blue', 'XL', 15.3, 'class2']
], columns=['color', 'size', 'price', 'classlabel'])

df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [2]:
df['x > M'] = df['size'].apply(lambda x: 1 if x in {'L', 'XL'} else 0)
df['x > L'] = df['size'].apply(lambda x: 1 if x == 'XL' else 0)
del df['size']
df

,color,price,classlabel,x > M,x > L
0,green,10.1,class2,0,0
1,red,13.5,class1,1,0
2,blue,15.3,class2,1,1


## 4.3. 데이터셋을 훈련 데이터셋과 테스트 데이터셋으로 나누기

머신러닝 모델을 훈련시키고 평가하기 위해서는 데이터셋을 훈련 데이터셋과 테스트 데이터셋으로 나누어야 한다. 훈련 데이터셋은 모델을 훈련시키는데 사용되고, 테스트 데이터셋은 모델을 평가하는데 사용된다.
